In [68]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
%matplotlib inline

from make_map import make_map

from file_util import load_matlab_data
from channel_overlap import *
from plot import plot_discharge
from metrics import *
from stratigraphy import *
from spatial_dist import *
from profiles import *
from make_pretty_images import *

  
import cPickle as pickle

In [2]:
TH_TOPO = -0.5
TH_ANGLE = 75
TH_VEL = 0.3 # use the threshold value for mud deposition (U_dep_mud)

fbase = 'baseF25data'
froot = '../JGR2016_data/'
sroot = 'mapfiles/'

In [20]:
# for individual datasets

for fnum in range(4099,4100):

    fname = fbase + str(fnum)
    data = load_matlab_data(froot, fname)

    topo = data['eta']
    uw = data['uw']
    
    
    '''
    Metrics calculated from the raw model output data
    '''
    
    # pretty RGB image showing topo in gray with water in blue
    topo_IMG = water_RGB_image(data['eta'], data['uw'], data['qw'])
    
    # creates colormap 'eta_cmap' good for plotting model topography
    # use like any other colormap
    elevation_colormap()
    
    # histograms of quantities
    bins, hists, cumhist, masks = quantity_histograms(topo,
                                                      min_bin = -3, max_bin = 1, dbin = 0.025,
                                                      radius1 = 3, radius2 = 30, radius3 = 90,
                                                      angle = np.pi/6.)
    
    
    # elevation profile at certain radii from source
    rad_profiles = radial_profile(data['eta'], [0.25, 0.5, 0.75])

    # longitudinal profile along main channel
    path_IMG, distance, long_profile = main_channel_path(data['eta'], threshold_value = 1)

    # get slices of topography (planview) at certain heights
    slices = slice_topography(data['eta'], [0, 0.25, 0.5])
    
    # total sediment volumes with optional mask
    dx = 100
    dz = 0.005 * 5 # spacing between saved strata
    mask = np.ones_like(topo)
    mask[:,:120] = 0

    vol_total, vol_sand, frac_sand = sediment_volumes(data['strata'], dx, dz, mask = mask)
    
    # sediment characteristics at certain radii from source
    seds = sedimentograph(data['strata'], step_radius = 10)
    
    # vertical slices of stratigraphy at certain radii from source
    strata_IMGs = radial_stratigraphy(data['strata'], radii = [0.3, 0.5])
    
    # sand body properties
    sand_corrL, sand_corrW = sand_body_properties(strata_IMGs[0])
    
    break
    
    
    
    
    # clip to make the file smaller
    topo_clip = topo[3:-5,5:-10]
    vel_clip = uw[3:-5,5:-10]

    mapfile = make_map(topo_clip,
                       vel_clip,
                       topo_threshold = TH_TOPO,
                       angle_threshold = TH_ANGLE,
                       velocity_threshold = TH_VEL,
                       numviews = 5,
                       save_file = True,
                       sroot = sroot,
                       fname = fname)
    
    '''
    Metrics calculated from the derived maps
    '''
    
    # areas and fractional areas
    frac_areas = fractional_areas(mapfile)
    
    # fractal dimension of channel network
    D_frac = fractal_dimension(mapfile['centerlinemap'])
    
    # island properties and map of edge distances
    island_IMG, island_props, ecdf, edgedist_IMG, edgedist_hist = island_properties(mapfile,
                                                                              smooth = True,
                                                                              properties = True)
    
    
    


Created new colormap 'eta_cmap'


In [69]:
from scipy import signal

fname = fbase + str(4100)
data = load_matlab_data(froot, fname)

strata_IMGs = radial_stratigraphy(data['strata'], radii = [0.5], density = 1.5)

In [54]:
sand_body_properties(strata_IMG)

(1000,)

In [63]:
# for multiple datasets

# map of changes in channel position over time
filenames = [sroot + fbase + str(i) + '.p' for i in range(4005,4010)]
overlap_IMG, overlap_diff, phi, O_phi, f_R = channel_overlap(filenames)

In [ ]:
# image of superimposed channels over time
channel_overlay_IMG = channel_overlay(froot, fbase, (4010,4012))

In [ ]:
# correlation of channel position over time
decay = channel_decay(froot, fbase, (4050,4100), number_of_comparisons = 50, average = True)

# curve fits for decay data
exponential = channel_decay_curve_fit(decay, fit = 'exponential')
harmonic = channel_decay_curve_fit(decay, fit = 'harmonic')
linear = channel_decay_curve_fit(decay, fit = 'linear')

plt.plot(exponential['xdata'], decay, 'ko')
plt.plot(exponential['xdata'], exponential['fit'], 'r')
plt.plot(harmonic['xdata'], harmonic['fit'], 'b')
plt.plot(linear['xdata'], linear['fit'], 'g')

In [66]:
difference

array([   0.,  245.,  354.,  438.,  564.])